# Heterogeneous mixing

By default, the Summer compartmental model assumes that each person in the model comes into contact with every other person at the same rate (homogeneous mixing). This assumption is preserved even after stratification has been applied (by default), such that the force of infection calculation will be the same for all model strata implemented. This notebook considered the incorporation of heterogeneous mixing patterns. For example, children may tend to interact more with other children than they do with elderly people (this is termed with-like or assortative mixing). When considering the impact of control strategies, especially strategies targeting sub groups of people, it is likely to be important to consider these differences in contact patterns.

## Standard preliminaries
Before we get into the code for heterogeneous mixing, let's start off with some of our standard (or "boilerplate") code to get everything set up.

In [ ]:
# pip install the required packages if running in Colab
try:
  import google.colab
  IN_COLAB = True
  %pip install summerepi
except:
  IN_COLAB = False

In [ ]:
# Standard imports, plotting option and constant definition
from datetime import datetime, timedelta
from typing import List
import pandas as pd
import plotly.express as px
import numpy as np

from summer import CompartmentalModel
from summer import Stratification

pd.options.plotting.backend = "plotly"

COVID_BASE_DATE = datetime(2019, 12, 31)
region = "Malaysia"

In [ ]:
# Get a function to access the Malaysia data if running in Colab
if IN_COLAB:
    !wget https://raw.githubusercontent.com/monash-emu/AuTuMN/master/notebooks/capacity_building/malaysia/get_mys_data.py

import get_mys_data

In [ ]:
# ... and use it to get the actual data
df = get_mys_data.fetch_mys_data()
initial_population = get_mys_data.get_initial_population(region)
observations = get_mys_data.get_target_observations(df, region, "cases")

In [ ]:
# Define the model running period and convert to a numeric representation
start_date = datetime(2021, 1, 1)
end_date = start_date + timedelta(days=300)
start_date_int = (start_date - COVID_BASE_DATE).days
end_date_int = (end_date - COVID_BASE_DATE).days

In [ ]:
# Define a target set of observations to compare against our modelled outputs later
notifications_target = observations[start_date: end_date]["cases_new"]

## Loading matrices
Get a mixing matrix that we can use for Malaysia.
This is not actually empiric data for Malaysia, but is what we have been using
and has the right format.

In [ ]:
# Get the mixing matrix for Malaysia
if IN_COLAB:
    !wget https://raw.githubusercontent.com/monash-emu/AuTuMN/master/notebooks/capacity_building/malaysia/MYS_matrices.pkl

mixing_matrix = pd.read_pickle("MYS_matrices.pkl", compression='infer')
age_mixing_matrix = mixing_matrix["all_locations"]

In [ ]:
# Let's have a look at the matrix using plotly express
px.imshow(age_mixing_matrix)

## Model

### Define a model

In [ ]:
def build_unstratified_model(base_compartments: List[str], parameters: dict) -> CompartmentalModel:
    """
    Create a compartmental model, with the minimal compartmental structure needed to run and produce some sort of 
    meaningful outputs.
    
    Args:
        base_compartments: The names of the base (unstratified compartments)
        infectious_seed: The number of infectious persons to start the epidemic
        parameters: Flow parameters
    
    """

    model = CompartmentalModel(
        times=(parameters["start_time"], parameters["end_time"]),
        compartments=base_compartments,
        infectious_compartments=["I"],
        ref_date=COVID_BASE_DATE
    )

    infectious_seed = parameters["infectious_seed"]

    model.set_initial_population(distribution={"S": initial_population - infectious_seed, "E": 0, "I": infectious_seed})
    
    # Susceptible people can get infected
    model.add_infection_frequency_flow(
        name="infection", 
        contact_rate=parameters["contact_rate"], 
        source="S", 
        dest="E"
    )
    # Expose people transition to infected
    model.add_transition_flow(
        name="progression",
        fractional_rate=parameters["progression_rate"],
        source="E",
        dest="I",
    )

    # Infectious people recover
    model.add_transition_flow(
        name="recovery",
        fractional_rate=parameters["recovery_rate"],
        source="I",
        dest="R",
    )

    # Add an infection-specific death flow to the I compartment
    model.add_death_flow(name="infection_death", death_rate=parameters["death_rate"], source="I")

    # We will also request an output for the 'progression' flow
    model.request_output_for_flow("progressions", "progression")

    return model

### Age stratification process

In [ ]:
def get_age_stratification(compartments_to_stratify: List[str], age_mixing_matrix) -> Stratification:
    """
    Create a summer stratification object that stratifies all of the compartments into
    16 age groups, which are intended to represent 5-year age bands starting from
    0-4 years, and finishing with the 75+ age group.
    This is essentially adapting the model's age stratification approach to the format
    of the mixing matrix, which is a perfectly reasonable approach.
    
    Args:
        compartments_to_stratify: List of the compartments to stratify, which should be all the compartments
    Returns:
        A summer stratification object to represent age stratification (not yet applied)
    """
    
    # Create the stratification, just naming the age groups by their starting value
    strata = [str(start_age) for start_age in range(0, 80, 5)]
    strat = Stratification(name="age", strata=strata, compartments=compartments_to_stratify)
    
    # Add the mixing matrix to the stratification
    strat.set_mixing_matrix(age_mixing_matrix)

    return strat

### Combine the two processes together

In [ ]:
def build_stratified_model(
    base_compartments: List[str], 
    parameters: dict, 
    age_mixng_matrix: np.ndarray,
) -> CompartmentalModel:
    """
    Get the model object with the age stratification with heterogeneous
    mixing applied to it.
    
    Arguments:
        base_compartments: The names of the compartments to be implemented
        parameters: A dictionary containing the parameter values to use
        age_mixing_matrix: 
        
    Returns:
        The model object        
    """
    
    # Get an unstratified model object
    model = build_unstratified_model(base_compartments, parameters)

    # Get and apply the stratification
    age_strat = get_age_stratification(base_compartments, age_mixing_matrix)
    model.stratify_with(age_strat)

    return model

### Outputs

In [ ]:
# Build and run the stratified model with some arbitrary parameters
parameters = {
    "contact_rate": 0.1,
    "progression_rate": 0.2,
    "recovery_rate": 0.2,
    "death_rate": 0.,
    "reporting_fraction": 0.01,
    "start_time": start_date_int,
    "end_time": end_date_int,
    "infectious_seed": 100.
}
base_compartments = ["S", "E", "I", "R"]

model = build_stratified_model(base_compartments, parameters, age_mixing_matrix)
model.run()

In [ ]:
progressions = model.get_derived_outputs_df()["progressions"]
notifications_modelled = progressions * parameters["reporting_fraction"]

In [ ]:
pd.DataFrame(
    {
        "observed": notifications_target,
        "modelled": notifications_modelled
    }
).plot()

## Manual calibration to Delta wave notifications

In [ ]:
# Define the model integration time to capture the Delta wave
start_date = datetime(2021, 1, 5)

model_duration = 290
end_date = start_date + timedelta(days=model_duration)

# Convert our start and end times to a numeric representation
start_date_int = (start_date - COVID_BASE_DATE).days
end_date_int = (end_date - COVID_BASE_DATE).days

# Define a set of parameters for this run
custom_params = {
        "contact_rate": 0.05,
        "progression_rate": 0.2,
        "recovery_rate": 0.2,
        "death_rate": 0.01,
        "reporting_fraction": 0.095,
        "start_time": start_date_int,
        "end_time": end_date_int,
        "infectious_seed": 100.
    }

# Run using our custom parameters
model = build_stratified_model(base_compartments, custom_params, age_mixing_matrix)
model.run()

# Get the outputs
progressions = model.get_derived_outputs_df()["progressions"]
notifications_modelled = progressions * custom_params["reporting_fraction"]

notifications_target = observations[start_date: end_date]["cases_new"]

pd.DataFrame(
    {
        "observed": notifications_target,
        "modelled": notifications_modelled
    }
).plot()